In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def prime_factors(n):
    F=factor(n)
    f0=[f[0] for f in F]
    return sortAndSmash(f0)

def omega(n):
    return len(prime_factors(n))

def listmap(f,lst):return list(map(f,lst))

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay1")

okay1


In [8]:
for n in [1..5]:
    print((n,sigma(n,k=11)))

(1, 1)
(2, 2049)
(3, 177148)
(4, 4196353)
(5, 48828126)


In [5]:
omegaSrs=0
for n in [1..10]:
    omegaSrs=omegaSrs+omega(n+1)*x^n
print(omegaSrs)
print(omegaSrs.truncate(4))

x^10 + 2*x^9 + x^8 + x^7 + x^6 + 2*x^5 + x^4 + x^3 + x^2 + x
x^3 + x^2 + x


In [3]:
omegaSrs=0
for n in [1..10000]:
    omegaSrs=omegaSrs+omega(n)*x^(n-1)

import time

def recip_omega(n):
    g=L(1/omegaSrs.truncate(2*n)
    return g

def recip_omegaPowerConstant(n):
    f=recip_omega(n)^n
    return f[0]


L=LaurentSeriesRing(QQ,'x', default_prec=10000)
import pickle
datafile= open('/Users/barrybrent/run14may23no2.txt','wb') 
data=[]
start=time.time()/3600
for a in [0..2000]:
    c=recip_omegaPowerConstant(a)
    data=data+[(a,c)]
    finish=time.time()/3600
    print((a,finish-start))
d = pickle.dumps(str(data))
datafile.write(d)
datafile.close()

(0, 1.8045306205749512e-05)
(1, 0.0003934316337108612)
(2, 0.009254783391952515)
(3, 0.03027578443288803)
(4, 0.058813367038965225)
(5, 0.10267703235149384)
(6, 0.14538214728236198)
(7, 0.20339079946279526)
(8, 0.24308384954929352)
(9, 0.29789484664797783)
(10, 0.3529840484261513)
(11, 0.4226178452372551)
(12, 0.4777536168694496)
(13, 0.5469999648630619)
(14, 0.6168510131537914)
(15, 0.7015589959919453)
(16, 0.7531885504722595)
(17, 0.8208643347024918)
(18, 0.8876831158995628)
(19, 0.9701755307614803)
(20, 1.0380484126508236)
(21, 1.1209581643342972)
(22, 1.2036769315600395)
(23, 1.3040264323353767)
(24, 1.3709785789251328)
(25, 1.4536778666079044)
(26, 1.5357702635228634)
(27, 1.6348626464605331)
(28, 1.7186799682676792)
(29, 1.820099014788866)
(30, 1.9203462153673172)
(31, 2.0362364314496517)
(32, 2.1050290651619434)
(33, 2.19078104570508)
(34, 2.276286616921425)
(35, 2.3724098205566406)
(36, 2.4520499631762505)
(37, 2.5463266372680664)
(38, 2.6409369967877865)
(39, 2.750441733747720

(326, 41.63119700178504)
(327, 41.79236253350973)
(328, 41.92326357960701)
(329, 42.06871332973242)
(330, 42.21445971727371)
(331, 42.37498276308179)
(332, 42.52178616821766)
(333, 42.683020282536745)
(334, 42.845357201993465)
(335, 43.02095792815089)
(336, 43.15114716440439)
(337, 43.297157764434814)
(338, 43.44296571612358)
(339, 43.604270700365305)
(340, 43.751374900341034)
(341, 43.9130492657423)
(342, 44.07412746548653)
(343, 44.250714868307114)
(344, 44.39583961293101)
(345, 44.557384848594666)
(346, 44.718640334904194)
(347, 44.89715361967683)
(348, 45.05874011665583)
(349, 45.23401214554906)
(350, 45.4102989025414)
(351, 45.6006912663579)
(352, 45.73098086193204)
(353, 45.878112595528364)
(354, 46.02712897956371)
(355, 46.19148766994476)
(356, 46.337264996021986)
(357, 46.497832383960485)
(358, 46.65860151126981)
(359, 46.83479966595769)
(360, 46.981033246964216)
(361, 47.14257816597819)
(362, 47.30451488494873)
(363, 47.48045701533556)
(364, 47.6404192969203)
(365, 47.81732865

(654, 96.19453676789999)
(655, 96.38240854814649)
(656, 96.52647578716278)
(657, 96.6846192330122)
(658, 96.84318508207798)
(659, 97.01590738072991)
(660, 97.17441321536899)
(661, 97.34761061519384)
(662, 97.52002263441682)
(663, 97.71041683107615)
(664, 97.86986131593585)
(665, 98.04269536584616)
(666, 98.21576840057969)
(667, 98.40377116575837)
(668, 98.57695110142231)
(669, 98.76588404923677)
(670, 98.95465428009629)
(671, 99.1577225998044)
(672, 99.30072711408138)
(673, 99.46019828692079)
(674, 99.61877501383424)
(675, 99.7927770987153)
(676, 99.95239286497235)
(677, 100.12617626413703)
(678, 100.29957833513618)
(679, 100.4864483140409)
(680, 100.6444358304143)
(681, 100.81787209585309)
(682, 100.99097327888012)
(683, 101.18151958286762)
(684, 101.35498944669962)
(685, 101.543127913028)
(686, 101.73133004829288)
(687, 101.93430867791176)
(688, 102.0932044647634)
(689, 102.26681188121438)
(690, 102.44149913266301)
(691, 102.62985903397202)
(692, 102.80369024723768)
(693, 102.9912491

(974, 157.66159933060408)
(975, 157.88283371180296)
(976, 158.06118473410606)
(977, 158.2513337470591)
(978, 158.44032673165202)
(979, 158.6489203684032)
(980, 158.83832456544042)
(981, 159.0424487479031)
(982, 159.24672643095255)
(983, 159.4672467149794)
(984, 159.6558213494718)
(985, 159.85967219993472)
(986, 160.06420383229852)
(987, 160.28307174518704)
(988, 160.48779236525297)
(989, 160.70593002811074)
(990, 160.92573734745383)
(991, 161.1620818786323)
(992, 161.33730403333902)
(993, 161.5266447328031)
(994, 161.71623073145747)
(995, 161.921969845891)
(996, 162.11045006290078)
(997, 162.31414219737053)
(998, 162.51812130212784)
(999, 162.73865493386984)
(1000, 162.92782663181424)
(1001, 163.13118528202176)
(1002, 163.3343208655715)
(1003, 163.55214210227132)
(1004, 163.75646198168397)
(1005, 163.9748187996447)
(1006, 164.19374631345272)
(1007, 164.42808077856898)
(1008, 164.61816485226154)
(1009, 164.82065109536052)
(1010, 165.0239947475493)
(1011, 165.2427179813385)
(1012, 165.44

(1283, 216.46555166691542)
(1284, 216.622973613441)
(1285, 216.79612005129457)
(1286, 216.96811094880104)
(1287, 217.15698554739356)
(1288, 217.31456803157926)
(1289, 217.48672346398234)
(1290, 217.65902709960938)
(1291, 217.8468551337719)
(1292, 218.01719683408737)
(1293, 218.2042201347649)
(1294, 218.392418961972)
(1295, 218.59402039647102)
(1296, 218.75161238387227)
(1297, 218.9237712174654)
(1298, 219.0971268005669)
(1299, 219.2838981859386)
(1300, 219.45822298526764)
(1301, 219.64586479961872)
(1302, 219.8331224322319)
(1303, 220.03590713068843)
(1304, 220.20793556421995)
(1305, 220.39480765163898)
(1306, 220.58163435012102)
(1307, 220.78332036733627)
(1308, 220.97112671285868)
(1309, 221.17831148207188)
(1310, 221.37998885288835)
(1311, 221.59648969769478)
(1312, 221.75354991480708)
(1313, 221.9243931993842)
(1314, 222.09492968022823)
(1315, 222.28170171752572)
(1316, 222.45366129651666)
(1317, 222.63985649868846)
(1318, 222.8280346803367)
(1319, 223.03037224709988)
(1320, 223.20

(1592, 278.1758242622018)
(1593, 278.38350646570325)
(1594, 278.5910585820675)
(1595, 278.8135693669319)
(1596, 279.0215559490025)
(1597, 279.2461405657232)
(1598, 279.4689526669681)
(1599, 279.7093140296638)
(1600, 279.87383659929037)
(1601, 280.052388433367)
(1602, 280.23067935183644)
(1603, 280.42364232987165)
(1604, 280.600813832134)
(1605, 280.7950149960816)
(1606, 280.9891332015395)
(1607, 281.1990794464946)
(1608, 281.3780849687755)
(1609, 281.5719057135284)
(1610, 281.7673037163913)
(1611, 281.97631046175957)
(1612, 282.1704341173172)
(1613, 282.3786314986646)
(1614, 282.5866065323353)
(1615, 282.81207229942083)
(1616, 282.99065501615405)
(1617, 283.18563088029623)
(1618, 283.38189454749227)
(1619, 283.5907413363457)
(1620, 283.7852967530489)
(1621, 283.9936689324677)
(1622, 284.20175439864397)
(1623, 284.4265651628375)
(1624, 284.62157333269715)
(1625, 284.83179952949286)
(1626, 285.040368296206)
(1627, 285.26353899762034)
(1628, 285.4716169349849)
(1629, 285.6951775290072)
(1

(1903, 346.88301445171237)
(1904, 347.0923896804452)
(1905, 347.31802041456103)
(1906, 347.54084346443415)
(1907, 347.7818275988102)
(1908, 348.0058989301324)
(1909, 348.24331196397543)
(1910, 348.4826016165316)
(1911, 348.741542365402)
(1912, 348.9662273824215)
(1913, 349.2048073336482)
(1914, 349.4450347274542)
(1915, 349.6989811472595)
(1916, 349.9403186812997)
(1917, 350.1940656155348)
(1918, 350.4479317702353)
(1919, 350.71703231707215)
(1920, 350.8973958976567)
(1921, 351.09244441241026)
(1922, 351.28835014626384)
(1923, 351.49661733210087)
(1924, 351.6904911994934)
(1925, 351.8999358937144)
(1926, 352.1089739985764)
(1927, 352.3340260013938)
(1928, 352.5278722830117)
(1929, 352.7385115996003)
(1930, 352.9484636671841)
(1931, 353.1724253632128)
(1932, 353.3821702301502)
(1933, 353.60669234767556)
(1934, 353.8312408477068)
(1935, 354.0702204480767)
(1936, 354.26609566807747)
(1937, 354.47656551748514)
(1938, 354.6860087327659)
(1939, 354.91259986907244)
(1940, 355.1219507008791)
(